In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

matplotlib.use("Qt5Agg")

In [3]:
def preprocess(df):
    df['width'] = df['circuit'].apply(lambda x: int(x.split('-')[0].strip('q')))
    df['depth'] = df['circuit'].apply(lambda x: int(x.split('-')[1].split('.')[0]))
    df.drop(columns=['circuit'], inplace=True)
    return df

stl = preprocess(pd.read_csv("linux_single_results.csv"))
mtl = preprocess(pd.read_csv("linux_mthread_results.csv"))

In [4]:
stl.describe()

,contraction_ns,contracted_exe_ns,normal_exe_ns,width,depth
count,1.500000e+02,1.500000e+02,1.500000e+02,150.000000,150.000000
mean,9.822120e+06,1.132362e+09,2.458162e+09,6.000000,255.000000
std,6.400489e+06,1.652421e+09,3.671339e+09,0.819232,144.792144
min,1.984740e+05,1.066288e+06,2.103459e+06,5.000000,10.000000
25%,4.681217e+06,1.408855e+08,1.030199e+08,5.000000,130.000000
50%,9.277949e+06,4.922842e+08,6.424230e+08,6.000000,255.000000
75%,1.372803e+07,1.241295e+09,3.427950e+09,7.000000,380.000000
max,2.783427e+07,7.562061e+09,1.322357e+10,7.000000,500.000000


In [5]:
from sklearn.linear_model import LinearRegression

def linreg(x, y):
    reg = LinearRegression().fit(x.values.reshape(-1, 1), y.values)
    xnew = np.linspace(0, max(x), 20)
    ynew = reg.coef_ * xnew + reg.intercept_
    ynew[ynew < 0] = 0
    print(f"Linear regression R^2: {reg.score(x.values.reshape(-1, 1), y.values)}")
    print(f"Linear regression slope: {reg.coef_}")
    print(f"Linear regression intercept: {reg.intercept_}")
    return xnew, ynew

def polyreg(x, y, degree):
    reg = np.polyfit(x, y, degree)
    xnew = np.linspace(0, max(x), 20)
    ynew = np.polyval(reg, xnew)
    print(f"Polynomial regression R^2: {np.corrcoef(np.polyval(reg, x), y)[0, 1] ** 2}")
    print(f"Polynomial regression coefficients (from highest to lowest degree): {reg}")
    return xnew, ynew

## Depth vs Contraction Time

In [6]:
x5 = stl[stl['width'] == 5]['depth']
x6 = stl[stl['width'] == 6]['depth']
x7 = stl[stl['width'] == 7]['depth']
y5 = stl[stl['width'] == 5]['contraction_ns'] / 1e6
y6 = stl[stl['width'] == 6]['contraction_ns'] / 1e6
y7 = stl[stl['width'] == 7]['contraction_ns'] / 1e6

# use sklearn to make a linear regression
x5new, y5new = linreg(x5, y5)
x6new, y6new = linreg(x6, y6)
x7new, y7new = linreg(x7, y7)

fig, ax = plt.subplots()

# give a title
ax.set_title('Contraction time vs circuit depth')
ax.set_xlabel('Circuit depth')
ax.set_ylabel('Contraction time (ms)')

ax.plot(x5, y5, 'x:', label='5 qubits', color='#1f77b4')
ax.plot(x6, y6, 'x:', label='6 qubits', color='#ff7f0e')
ax.plot(x7, y7, 'x:', label='7 qubits', color='#2ca02c')
ax.plot(x5new, y5new, label='5 qubits linear regression', color='#1f77b4')
ax.plot(x6new, y6new, label='6 qubits linear regression', color='#ff7f0e')
ax.plot(x7new, y7new, label='7 qubits linear regression', color='#2ca02c')

plt.legend()
plt.grid()
plt.show()

# save fig
fig.savefig('depth_vs_contraction_time.png', dpi=300)

Linear regression R^2: 0.9523278168086171
Linear regression slope: [0.02803263]
Linear regression intercept: -0.03433408816326633
Linear regression R^2: 0.8791064764214666
Linear regression slope: [0.04049617]
Linear regression intercept: -0.2225108334693875
Linear regression R^2: 0.9764635615449342
Linear regression slope: [0.04973185]
Linear regression intercept: -0.4332596669387794


In [7]:
x5 = stl[stl['width'] == 5]['depth']
x6 = stl[stl['width'] == 6]['depth']
x7 = stl[stl['width'] == 7]['depth']
y5 = stl[stl['width'] == 5]['contraction_ns'] / 1e6
y6 = stl[stl['width'] == 6]['contraction_ns'] / 1e6
y7 = stl[stl['width'] == 7]['contraction_ns'] / 1e6
z5 = stl[stl['width'] == 5]['contracted_exe_ns'] / 1e6
z6 = stl[stl['width'] == 6]['contracted_exe_ns'] / 1e6
z7 = stl[stl['width'] == 7]['contracted_exe_ns'] / 1e6

y5 /= z5
y6 /= z6
y7 /= z7

# use sklearn to make a linear regression
# x5new, y5new = polyreg(x5, y5, 5)
# x6new, y6new = polyreg(x6, y6, 5)
# x7new, y7new = polyreg(x7, y7, 5)

fig, ax = plt.subplots()

# give a title
ax.set_title('Contraction cost vs circuit depth')
ax.set_xlabel('Circuit depth')
ax.set_ylabel('Contraction time / Execution time (%)')

ax.plot(x5, y5, 'x-.', label='5 qubits', color='#1f77b4')
ax.plot(x6, y6, 'x-.', label='6 qubits', color='#ff7f0e')
ax.plot(x7, y7, 'x-.', label='7 qubits', color='#2ca02c')
# ax.plot(x5new, y5new, label='5 qubits linear regression', color='#1f77b4')
# ax.plot(x6new, y6new, label='6 qubits linear regression', color='#ff7f0e')
# ax.plot(x7new, y7new, label='7 qubits linear regression', color='#2ca02c')

plt.legend()
plt.grid()
plt.show()

# save fig
fig.savefig('depth_vs_contraction_cost.png', dpi=300)

## Depth vs Execution Time

In [15]:
f = lambda s: stl[stl['width'] == s]

x5 = f(5)['depth']
x6 = f(6)['depth']
x7 = f(7)['depth']
y5c = f(5)['contracted_exe_ns'] / 1e9
y5n = f(5)['normal_exe_ns'] / 1e9
y6c = f(6)['contracted_exe_ns'] / 1e9
y6n = f(6)['normal_exe_ns'] / 1e9
y7c = f(7)['contracted_exe_ns'] / 1e9
y7n = f(7)['normal_exe_ns'] / 1e9

# use sklearn to make a linear regression
x5cnew, y5cnew = linreg(x5, y5c)
x5nnew, y5nnew = linreg(x5, y5n)
x6cnew, y6cnew = linreg(x6, y6c)
x6nnew, y6nnew = linreg(x6, y6n)
x7cnew, y7cnew = linreg(x7, y7c)
x7nnew, y7nnew = linreg(x7, y7n)

fig, ax = plt.subplots()

# give a title
ax.set_title('Execution time vs circuit depth')
ax.set_xlabel('Circuit depth')
ax.set_ylabel('Execution time (s)')

ax.plot(x5, y5c, 'x-.', label='Contraction on 5q', color='#42c2f5')
ax.plot(x5, y5n, 'x-.', label='Normal on 5q', color='#4287f5')
ax.plot(x6, y6c, 'x-.', label='Contraction on 6q', color='#88f289')
ax.plot(x6, y6n, 'x-.', label='Normal on 6q', color='#34e036')
ax.plot(x7, y7c, 'x-.', label='Contraction on 7q', color='#ff7a38')
ax.plot(x7, y7n, 'x-.', label='Normal on 7q', color='#de5d3a')
# ax.plot(x5cnew, y5cnew, color='#42c2f5')
# ax.plot(x5nnew, y5nnew, color='#4287f5')
# ax.plot(x6cnew, y6cnew, color='#88f289')
# ax.plot(x6nnew, y6nnew, color='#34e036')
# ax.plot(x7cnew, y7cnew, color='#ff7a38')
# ax.plot(x7nnew, y7nnew, color='#de5d3a')

plt.legend()
plt.grid()
plt.show()

# save fig
fig.savefig('depth_vs_execution_time.png', dpi=300)

Linear regression R^2: 0.9163291971877449
Linear regression slope: [0.00164237]
Linear regression intercept: -0.14485057091346953
Linear regression R^2: 0.9908356413765276
Linear regression slope: [0.00029034]
Linear regression intercept: -0.0002843563542857114
Linear regression R^2: 0.9117431247555036
Linear regression slope: [0.00369616]
Linear regression intercept: -0.313216105892245
Linear regression R^2: 0.9941667678809559
Linear regression slope: [0.00276609]
Linear regression intercept: 0.004206584668571489
Linear regression R^2: 0.9042455249232457
Linear regression slope: [0.01461618]
Linear regression intercept: -1.2332998415085723
Linear regression R^2: 0.9947509646826481
Linear regression slope: [0.0260255]
Linear regression intercept: -0.04532804319755179


In [58]:
x5 = f(5)['depth']
x6 = f(6)['depth']
x7 = f(7)['depth']
y5 = f(5)['contracted_exe_ns']
y5 += f(5)['contraction_ns']
y5 /= f(5)['normal_exe_ns']
y6 = f(6)['contracted_exe_ns']
y6 += f(6)['contraction_ns']
y6 /= f(6)['normal_exe_ns']
y7 = f(7)['contracted_exe_ns']
y7 += f(7)['contraction_ns']
y7 /= f(7)['normal_exe_ns']
y5 = 1 / y5
y6 = 1 / y6
y7 = 1 / y7

# use sklearn to make a linear regression
x5new, y5new = polyreg(x5, y5, 4)
x6new, y6new = polyreg(x6, y6, 4)
x7new, y7new = polyreg(x7, y7, 4)

fig, ax = plt.subplots()

# give a title
ax.set_title('Speedup vs circuit depth')
ax.set_xlabel('Circuit depth')
ax.set_ylabel('Speedup (times)')

ax.plot(x5, y5, 'x:', label='Speedup on 5q', color='#4287f5')
ax.plot(x6, y6, 'x:', label='Speedup on 6q', color='#34e036')
ax.plot(x7, y7, 'x:', label='Speedup on 7q', color='#de5d3a')
ax.plot(x5new, y5new, label='poly. regr. 5q', color='#4287f5')
ax.plot(x6new, y6new, label='poly. regr. 6q', color='#34e036')
ax.plot(x7new, y7new, label='poly. regr. 7q', color='#de5d3a')

# add a bold red horizontal line at y=1
plt.axhline(y=1, color='r', linestyle='--')
# and comment it
plt.text(110, 1.2, 'no speedup', color='r')
# # color in green all y > 1
# plt.fill_between([0, 500], 1, 10, color='green', alpha=0.1)
# # color in red all y < 1
# plt.fill_between([0, 500], 0, 1, color='red', alpha=0.1)

plt.legend()
plt.grid()
plt.show()

# save fig
fig.savefig('depth_vs_execution_speedup.png', dpi=300)

Polynomial regression R^2: 0.9474947580295692
Polynomial regression coefficients (from highest to lowest degree): [ 2.17481223e-10 -2.63704137e-07  1.15102608e-04 -2.25138753e-02
  2.09596338e+00]
Polynomial regression R^2: 0.9788212343777231
Polynomial regression coefficients (from highest to lowest degree): [ 2.05890335e-10 -2.67078669e-07  1.32768276e-04 -3.40359027e-02
  5.12967916e+00]
Polynomial regression R^2: 0.9041989815950865
Polynomial regression coefficients (from highest to lowest degree): [-9.90373558e-11  1.17701817e-07 -2.54661076e-05 -1.72705968e-02
  8.24292313e+00]


## Depth vs (Execution Time difference)

In [13]:
x5 = f(5)['depth']
x6 = f(6)['depth']
x7 = f(7)['depth']
y5 = f(5)['normal_exe_ns'] / 1e9 - f(5)['contracted_exe_ns'] / 1e9
y6 = f(6)['normal_exe_ns'] / 1e9 - f(6)['contracted_exe_ns'] / 1e9
y7 = f(7)['normal_exe_ns'] / 1e9 - f(7)['contracted_exe_ns'] / 1e9

# use sklearn to make a linear regression
x5new, y5new = polyreg(x5, y5, 2)
x6new, y6new = polyreg(x6, y6, 2)
x7new, y7new = polyreg(x7, y7, 2)

fig, ax = plt.subplots()

# give a title
ax.set_title('Time gained vs circuit depth')
ax.set_xlabel('Circuit depth')
ax.set_ylabel('Time gained (s)')

ax.plot(x5, y5, 'x:', label='5 qubits', color='#1f77b4')
ax.plot(x6, y6, 'x:', label='6 qubits', color='#ff7f0e')
ax.plot(x7, y7, 'x:', label='7 qubits', color='#2ca02c')
ax.plot(x5new, y5new, label='5 qubits linear regression', color='#1f77b4')
ax.plot(x6new, y6new, label='6 qubits linear regression', color='#ff7f0e')
ax.plot(x7new, y7new, label='7 qubits linear regression', color='#2ca02c')

plt.legend()
plt.grid()
plt.show()

# save fig
fig.savefig('depth_vs_time_gained.png', dpi=300)

Polynomial regression R^2: 0.9814055470278802
Polynomial regression coefficients (from highest to lowest degree): [-3.41692584e-06  3.90596527e-04 -6.46190774e-03]
Polynomial regression R^2: 0.9586428671412802
Polynomial regression coefficients (from highest to lowest degree): [-8.04126024e-06  3.17097798e-03 -3.80010121e-02]
Polynomial regression R^2: 0.9809737066218605
Polynomial regression coefficients (from highest to lowest degree): [-2.97579493e-05  2.65858753e-02 -1.27329562e-01]
